<a href="https://colab.research.google.com/github/massivetexts/llm_aut_study/blob/main/notebooks/Process_AUT_GT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook collects, normalizes and processes AUT human judged ground truth from various sources. This is the data used in [Beyond Semantic Distance: Automated Scoring of Divergent Thinking Greatly Improves with Large Language Models](https://www.researchgate.net/publication/363456838_Beyond_Semantic_Distance_Automated_Scoring_of_Divergent_Thinking_Greatly_Improves_with_Large_Language_Models/stats).

This is an assortment of studies, with different demographics, goals, and test setups. It is most appropriate for supervised learning of automated scoring, where we're not necessarily trying to learn about the participants but the raters in general.

In [1]:
import dataclasses
#@title import and setup defs
import pandas as pd
import os
import numpy as np
import hashlib
from pathlib import Path
from sklearn.model_selection import train_test_split
import json

!pip install --upgrade xlrd
!pip -qq install pingouin
import pingouin as pg

def simple_stats(df, rater_cols=False):
    print("# of prompts", len(df.prompt.unique()))
    print("# of participants", len(df.participant.unique()))
    print("# of data points", len(df))
    print("Prompts", df.prompt.unique())
    if rater_cols:
        print("# of raters", len(rater_cols))
        print("Intraclass correlation coefficients (report ICC2k)")
        x = df[['id']+rater_cols].melt(id_vars='id', var_name='rater', value_name='rating')
        icc = pg.intraclass_corr(data=x, targets='id', raters='rater', ratings='rating', nan_policy='omit')
        display(icc.round(2))

include_rater_std = False #@param {type:'boolean'}
returncols = ['type', 'src', 'question', 'prompt', 'response', 'id', 'target', 'participant', 'response_num']
if include_rater_std:
    returncols += ['rating_std']

def prep_general(data, src, rater_cols=None, return_full=False,
                 aggregate_scores=True,
                 drop_noresponse=True, include_stats=True, round_adjust=False,
                 include_rater_std=False, overwrite_q=False, inputrange=None):
    ''' General cleaning that repeats for multiple datasets.

    If aggregate_scores is False, this will (eventually) return original individual rater scores, rather than multi-rater.
        I haven't implemented this disagregation yet because I'm not sure how well disagregated
        scored could be used in a proper training, without data leakage.
    '''
    if not rater_cols:
        # assume columns that say 'rater'
        rater_cols = [col for col in data.columns if 'rater' in col.lower()]

    print("Rater cols:", rater_cols)

    if drop_noresponse:
        data = data[~data.response.isna()]

    data['src'] = src
    data['avg_rating'] = data[rater_cols].mean(1)
    if round_adjust:
        # add a tiny bit to avg rater to round in direction of median. Only need if there are tiebreakers
        data['median_rating'] = data[rater_cols].median(1)
        data['avg_rating'] = data.avg_rating + (data.median_rating - data.avg_rating).div(10**3)
    if include_rater_std:
        data['rating_std'] = data[rater_cols].std(1)

    data['target'] = normalize_values(data.avg_rating, oldrange=inputrange)

    missing = data.target.isna()
    if missing.sum():
        print(f'Dropping {missing.sum()} unrated items')
        data = data[~missing]

    data['participant'] = src + data['participant'].astype(str)

    if 'type' not in data.columns:
        print('assuming type="uses"')
        data['type'] = 'uses'

    if ('question' not in data.columns) or overwrite_q:
        try:
            assert data.type.unique().tolist() == ['uses']
        except:
            raise Exception("can't infer question format for anything other than uses; please explicitly supply a 'question' column")
        data.loc[data.type=='uses', 'question'] = data.loc[data.type=='uses', 'prompt'].apply(lambda x: f"What is a surprising use for a {x.upper()}?")

    idhash = (data.participant+data.response).apply(lambda x: hashlib.md5(x.encode('utf-8')).hexdigest()[:6])
    # task can be a custom string that is used instead of prompt (e.g. 'g2_red' instead of 'red')
    data['id'] = f'{src}_' + data['task' if 'task' in data.columns else 'prompt'].astype(str) + '-' + idhash

    if include_stats:
        simple_stats(data, rater_cols)

    if return_full:
        return data
    else:
        cols = ['type', 'src', 'question', 'prompt', 'response', 'id', 'target', 'participant', 'response_num']
        if include_rater_std:
            cols += ['rating_std']
        return data[cols]

datasets = dict()

c:\Users\Maxime\anaconda3\envs\ml\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [ ]:
#base_dir = 'drive/MyDrive/Grants/MOTES/Data/aut_ground_truth' #@param { type: 'string' }
#base_dir = Path(base_dir)

In [ ]:
# All AUT ICC2ks, to get a sense of reliability
#x = np.array([0.85, 0.69, 0.79, 0.48, 0.67, 0.81, 0.67, 0.72])
#np.mean(x), np.std(x).round(3), np.median(x)

In [2]:
#@title Normalization Settings
#@markdown Ensure that all the datasets have the same scale
scale_min = 1 #@param{type:'number'}
scale_max = 5 #@param{type:'number'}
#@markdown Round to nearest 0.1 by default.
round_func = lambda x: x.round(1) #@param ["lambda x: x.multiply(2).round(0).div(2)", "None", "lambda x: x.round(1)"] {type:"raw"}

def normalize_values(series, outrange=(1,5), oldrange=None,
                     round_func=round_func):
    min, max = outrange
    if oldrange:
        oldmin, oldmax = oldrange
    else:
        oldmin, oldmax = series.min(), series.max()

    x = (series - oldmin)/(oldmax-oldmin)
    x = min + (max-min)*x
    if round_func:
        # none of our data has even numbers of raters, so there shouldn't be need for rounding tiebreakers
        # but for future proofing, add or subtract a tiny number randomly, so that rounding can go either way
        modifier = (2*np.random.randint(0,2,size=x.shape)-1)/10**5
        x = round_func(x+modifier)
    return x



## Normalize Dumas et al 2020

Download from OSF.

In [6]:
#!wget -q -O dod20.csv https://osf.io/download/u3yv4/

src = 'dod20' 
paca = pd.read_csv('./../data/dod20.csv', index_col=0)
paca = paca[~paca.response.str.contains('!!!')]
datasets[src] = prep_general(paca, src, include_rater_std=include_rater_std)
datasets[src].sample(2)

Rater cols: ['rater1', 'rater2', 'rater3', 'rater4']
assuming type="uses"
# of prompts 10
# of participants 92
# of data points 5490
Prompts ['book' 'bottle' 'brick' 'fork' 'pants' 'rope' 'shoe' 'shovel' 'table'
 'tire']
# of raters 4
Intraclass correlation coefficients (report ICC2k)


,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.57,6.27,5376,16131,0.0,"[0.56, 0.58]"
1,ICC2,Single random raters,0.58,7.78,5376,16128,0.0,"[0.49, 0.65]"
2,ICC3,Single fixed raters,0.63,7.78,5376,16128,0.0,"[0.62, 0.64]"
3,ICC1k,Average raters absolute,0.84,6.27,5376,16131,0.0,"[0.83, 0.85]"
4,ICC2k,Average random raters,0.85,7.78,5376,16128,0.0,"[0.79, 0.88]"
5,ICC3k,Average fixed raters,0.87,7.78,5376,16128,0.0,"[0.87, 0.88]"


,type,src,question,prompt,response,id,target,participant,response_num
4180,uses,dod20,What is a surprising use for a TIRE?,tire,gravel substitute,dod20_tire-30e5e6,3.0,dod2061,0
3445,uses,dod20,What is a surprising use for a FORK?,fork,hair straightener,dod20_fork-925d45,3.7,dod2048,0


## Process Silvia et 2009

Silvia, P. J., Nusbaum, E. C., Berg, C., Martin, C., & O'Connor, A. (2009). Openness to experience, plasticity, and creativity: Exploring lower-order, high-order, and interactive effects. Journal of Research in Personality, 43(6), 1087–1090. https://doi.org/10.1016/j.jrp.2009.04.015

In [7]:
#!wget -q -O silvia09.csv https://osf.io/download/qdrv8/
src = 'snbmo09'
data = pd.read_csv('./../data/silvia09.csv').rename(columns={'subject':'participant','response_order':'response_num'})
data['prompt'] = data.task.apply(lambda x: x.split('_')[-1])
datasets[src] = prep_general(data, src, include_rater_std=include_rater_std)

Rater cols: ['rater_1', 'rater_2', 'rater_3', 'rater_4']
Dropping 10 unrated items
assuming type="uses"
# of prompts 3
# of participants 202
# of data points 4099
Prompts ['brick' 'knife' 'box']
# of raters 4
Intraclass correlation coefficients (report ICC2k)


,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.33,2.97,4097,12294,0.0,"[0.31, 0.35]"
1,ICC2,Single random raters,0.36,4.10,4097,12291,0.0,"[0.25, 0.45]"
2,ICC3,Single fixed raters,0.44,4.10,4097,12291,0.0,"[0.42, 0.45]"
3,ICC1k,Average raters absolute,0.66,2.97,4097,12294,0.0,"[0.65, 0.68]"
4,ICC2k,Average random raters,0.69,4.10,4097,12291,0.0,"[0.57, 0.77]"
5,ICC3k,Average fixed raters,0.76,4.10,4097,12291,0.0,"[0.74, 0.77]"


In [8]:
datasets[src].sample(1)

,type,src,question,prompt,response,id,target,participant,response_num
131,uses,snbmo09,What is a surprising use for a BRICK?,brick,building,snbmo09_1_brick-455391,1.0,snbmo099,2


## Process Hass 2017

Hass, R. W. (2017). Semantic search during divergent thinking. Cognition, 166, 344–357. https://doi.org/10.1016/j.cognition.2017.05.039

Data at https://osf.io/ng598.

This study looked at uses for *bottle* and *brick*. There were 54 participants after data cleaning.

Rating was on a 5-point scale. For verification, their reported inter-rater reliability was ICC2k was 0.80 for brick and 0.78 for bottle, which is about what we see below.

The rating data was stoplisted, so I need to reconstruct the original responses here.

In [ ]:
#!wget -q -O hass17ratings.xslx https://osf.io/download/mcykr/ # rater scores
#!wget -q -O hass17responses1.xslx https://osf.io/download/27bx8/ # responses 1
#!wget -q -O hass17responses2.xslx https://osf.io/download/rzvyd/ # responses 2

In [16]:
src = 'hass17'
all_ratings = []
for sheet, prompt in [('br_exp1', 'brick'),('br_exp2', 'brick'),('bot_exp1', 'bottle'),('bot_exp2', 'bottle')]:
    data = pd.read_excel('./../data/hass17ratings.xlsx', sheet_name=sheet) #.rename(columns={'subject':'participant','response_order':'response_num'}) # typo here: xls instead of xlsx
    data['prompt'] = prompt
    all_ratings.append(data)
hassratings = pd.concat(all_ratings).rename(columns={'response':'cleaned'})
participants = pd.concat([pd.read_excel('./../data/hass17responses1.xls'), pd.read_excel('./../data/hass17responses2.xlsx')]) # typo here: xls instead of xlsx

In [17]:
# melt original responses to long, reconstructe the cleaned columns, then join with ratings
long_part = participants.melt(id_vars='ID', value_name='response').rename(columns={'ID':'participant'})
long_part = long_part[long_part.variable.str.contains('resp') & ~long_part.variable.str.contains('time')].dropna()
long_part[['prompt', 'response_num']] = long_part.variable.str.split('_', expand=True)
long_part.loc[long_part.prompt.str.contains('resp1'), 'prompt'] = 'bottle'
long_part.loc[long_part.prompt.str.contains('resp2'), 'prompt'] = 'brick'
long_part.sample(10)

import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stops = stopwords.words('english')
# not sure which list the study used, so just adjust based on testing
stops += ['could']
stops = [w for w in stops if w not in ['can']]
stops = set(stops)

def stop_clean(x):
    x = x.lower()
    x = x.replace('i.e.', 'e') # quirk of the tokenization in original study
    for c in list("/\\'-()"):
        x = x.replace(c, '')
    words = [word for word in word_tokenize(x) if word not in stops]
    return " ".join(words)

long_part['cleaned'] = long_part.response.apply(stop_clean)
hass07 = long_part.merge(hassratings, how='left', on=['prompt', 'cleaned'])
datasets[src] = prep_general(hass07, src, rater_cols=['r1','r2','r3'], include_rater_std=include_rater_std)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Maxime\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Maxime\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Rater cols: ['r1', 'r2', 'r3']
assuming type="uses"
# of prompts 2
# of participants 57
# of data points 1093
Prompts ['bottle' 'brick']
# of raters 3
Intraclass correlation coefficients (report ICC2k)


,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.55,4.68,794,1590,0.0,"[0.51, 0.59]"
1,ICC2,Single random raters,0.56,5.04,794,1588,0.0,"[0.5, 0.6]"
2,ICC3,Single fixed raters,0.57,5.04,794,1588,0.0,"[0.54, 0.61]"
3,ICC1k,Average raters absolute,0.79,4.68,794,1590,0.0,"[0.76, 0.81]"
4,ICC2k,Average random raters,0.79,5.04,794,1588,0.0,"[0.75, 0.82]"
5,ICC3k,Average fixed raters,0.80,5.04,794,1588,0.0,"[0.78, 0.82]"


## Process Silvia et al Data 2008

From https://osf.io/dh7ey, this was the order of creativity tasks:

1. Please list all of the creative, unusual uses for a brick that you can think of.
2. Please list all of the creative, unusual instances of things that are round that you can think of.
3. Imagine that people no longer needed to sleep. Please list creative, unusual consequences that would follow.
4. Please list all of the creative, unusual uses for a knife that you can think of.
5. Please list all of the creative, unusual instances of things that will make a noise that you can think of.
6. Imagine that everyone shrank to 12 inches tall. Please list creative, unusual consequences that would follow.

Numbers 1 and 4 are AUT.



In [20]:
#!pip -q install pyreadstat
#!wget -q -O SilvaStudy2.zip https://files.osf.io/v1/resources/4ketx/providers/osfstorage/5dd70d1f83135e000ec3c242/?zip=
#!unzip -o SilvaStudy2.zip
import pyreadstat

In [22]:
src = 'setal08'
promptref = ['brick', 'round', 'no sleep', 'knife', 'noise', 'shrank']
qref = ["What is a surprising use for a BRICK?",
        "What is a surprising thing that is ROUND?",
        "What would be a surprising consequence if PEOPLE NEEDED NO SLEEP?",
        "What is a surprising use for a KNIFE?",
        "What is a surprising thing that makes a NOISE?",
        "What would be a surprising consequence if EVERYONE SHRANK TO 12 INCHES TALL?"]
typeref = ['uses', 'instances', 'consequences', 'uses', 'instances', 'consequences']
qref_dict = dict(zip(promptref, qref))

df, meta = pyreadstat.read_sav('./../data/silvia_2008/DT_Responses_PACA_2008_Study_2.sav')
df = df.rename(columns={'subject':'participant', 'order':'response_num'})
df['prompt'] = df.task.apply(lambda x: promptref[int(x)-1])
df['type'] = df.task.apply(lambda x: typeref[int(x)-1])
df['participant'] = df['participant'].astype(int)
df['question'] = df.prompt.apply(lambda x: qref_dict[x])
df = prep_general(df, src, include_stats=True, return_full=True,
                  include_rater_std=include_rater_std)
# doublecheck - burczak reported ICC2k as 0.48 (for the s)
print("Testing stats just for AUT (burczak et al reported ICC2k as 0.48)")
prep_general(df.query("type=='uses'"), src, include_stats=True)
datasets[src] = df[returncols]
datasets[src].sample(2)

Rater cols: ['rater1', 'rater2', 'rater3']
Dropping 37 unrated items
# of prompts 6
# of participants 242
# of data points 11490
Prompts ['brick' 'round' 'no sleep' 'knife' 'noise' 'shrank']
# of raters 3
Intraclass correlation coefficients (report ICC2k)


,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.12,1.40,11488,22978,0.0,"[0.11, 0.13]"
1,ICC2,Single random raters,0.20,2.16,11488,22976,0.0,"[0.09, 0.3]"
2,ICC3,Single fixed raters,0.28,2.16,11488,22976,0.0,"[0.27, 0.29]"
3,ICC1k,Average raters absolute,0.29,1.40,11488,22978,0.0,"[0.26, 0.31]"
4,ICC2k,Average random raters,0.43,2.16,11488,22976,0.0,"[0.22, 0.57]"
5,ICC3k,Average fixed raters,0.54,2.16,11488,22976,0.0,"[0.52, 0.55]"


Testing stats just for AUT (burczak et al reported ICC2k as 0.48)
Rater cols: ['rater1', 'rater2', 'rater3']
# of prompts 2
# of participants 241
# of data points 3425
Prompts ['brick' 'knife']
# of raters 3
Intraclass correlation coefficients (report ICC2k)


,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.12,1.42,3424,6850,0.0,"[0.1, 0.15]"
1,ICC2,Single random raters,0.23,2.76,3424,6848,0.0,"[0.06, 0.39]"
2,ICC3,Single fixed raters,0.37,2.76,3424,6848,0.0,"[0.35, 0.39]"
3,ICC1k,Average raters absolute,0.30,1.42,3424,6850,0.0,"[0.26, 0.34]"
4,ICC2k,Average random raters,0.48,2.76,3424,6848,0.0,"[0.15, 0.65]"
5,ICC3k,Average fixed raters,0.64,2.76,3424,6848,0.0,"[0.62, 0.66]"


,type,src,question,prompt,response,id,target,participant,response_num
6721,uses,setal08,What is a surprising use for a KNIFE?,knife,pick a lock,setal08_4.0-9cc70e,2.0,setal08143,5.0
991,uses,setal08,What is a surprising use for a KNIFE?,knife,open a can,setal08_4.0-628b62,2.0,setal0823,2.0


## Process Hofelich Mohr, Sell, and Lindsay

Doublecheck ICC2k - buczak paper had icc2k=0.67

In [ ]:
#!wget -qO hofelich.zip https://conservancy.umn.edu/bitstream/handle/11299/172116/HMSL_CSV%20Data%20Files.zip?sequence=28&isAllowed=y
#!unzip -qo hofelich.zip

In [42]:
src = 'hmsl'
df = pd.read_csv(f'./../data/hofelich/HMSL_Originality_scores_all.csv')
df = df.rename(columns={'Item': 'prompt', 'QLogin_1':'participant'})
rater_cols = ['J1_Rating','J2_Rating','J3_Rating','J4_Rating']
# remove three ratings that are 11
df[rater_cols] = df[rater_cols].replace(11, pd.NA)

datasets[src] = prep_general(df, src, rater_cols=rater_cols, include_rater_std=include_rater_std)
datasets[src].sample(2)
df.head()

Rater cols: ['J1_Rating', 'J2_Rating', 'J3_Rating', 'J4_Rating']


C:\Users\Maxime\AppData\Local\Temp\ipykernel_21688\2386548369.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['src'] = src
C:\Users\Maxime\AppData\Local\Temp\ipykernel_21688\2386548369.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['avg_rating'] = data[rater_cols].mean(1)


TypeError: loop of ufunc does not support argument 0 of type float which has no callable rint method

## Process dataset used by Beaty and Johnson

From SemDis paper:

- Study 1 was re-analysis of AUT responses from Beaty et al., 2018 to see if ensemble approaches work better. Two tests: `box` and `rope`
   - according to their paper, using additive composition was slightly negative correlation, while multiplicative 'results revealed a large correlation between latent semantic distance and human ratings:$r=.91$, p<.001'. This uses a model that weighs the factors, but is (I think) tailored to the dataset without held out data.

- Study 2 was re-analysis of results from Silvia et al. 2017, also on box and rope
- Study 3 was brick - yet again - via Beaty and Silvia 2012
- Study 4 and 5- Heinen and Johnson (2018) - were noun matching, not relevant here

In [25]:
#!wget -qO beaty.zip https://files.osf.io/v1/resources/gz4fc/providers/osfstorage/5e45b6c73e86a800be6e662e/?zip=
#!unzip -qo beaty.zip
files = sorted(list(Path('.').glob('./../data/Data/Study*/*data_long.xlsx')))

In [26]:
for fname,src in zip(files[:3], ['betal18','snb17', 'bs12']):
    print(src.upper().center(30, '-'))
    df = pd.read_excel(fname)

    df = df.rename(columns={'id':'participant', 'item':'prompt'})
    df['response_num'] = None
    datasets[src] = prep_general(df, src, include_rater_std=include_rater_std)

-----------BETAL18------------
Rater cols: ['rater1', 'rater2', 'rater3', 'rater4']
assuming type="uses"
# of prompts 2
# of participants 171
# of data points 2918
Prompts ['box' 'rope']
# of raters 4
Intraclass correlation coefficients (report ICC2k)


,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.52,5.26,2894,8685,0.0,"[0.5, 0.53]"
1,ICC2,Single random raters,0.52,6.18,2894,8682,0.0,"[0.46, 0.58]"
2,ICC3,Single fixed raters,0.56,6.18,2894,8682,0.0,"[0.55, 0.58]"
3,ICC1k,Average raters absolute,0.81,5.26,2894,8685,0.0,"[0.8, 0.82]"
4,ICC2k,Average random raters,0.82,6.18,2894,8682,0.0,"[0.77, 0.85]"
5,ICC3k,Average fixed raters,0.84,6.18,2894,8682,0.0,"[0.83, 0.85]"


------------SNB17-------------
Rater cols: ['rater1', 'rater2', 'rater3']
assuming type="uses"
# of prompts 2
# of participants 142
# of data points 2372
Prompts ['box' 'rope']
# of raters 3
Intraclass correlation coefficients (report ICC2k)


,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.38,2.82,2352,4706,0.0,"[0.35, 0.4]"
1,ICC2,Single random raters,0.40,3.57,2352,4704,0.0,"[0.29, 0.5]"
2,ICC3,Single fixed raters,0.46,3.57,2352,4704,0.0,"[0.44, 0.49]"
3,ICC1k,Average raters absolute,0.65,2.82,2352,4706,0.0,"[0.62, 0.67]"
4,ICC2k,Average random raters,0.67,3.57,2352,4704,0.0,"[0.55, 0.75]"
5,ICC3k,Average fixed raters,0.72,3.57,2352,4704,0.0,"[0.7, 0.74]"


-------------BS12-------------
Rater cols: ['br_rater1', 'br_rater2', 'br_rater3']
assuming type="uses"
# of prompts 1
# of participants 133
# of data points 1807
Prompts ['brick']
# of raters 3
Intraclass correlation coefficients (report ICC2k)


,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.43,3.24,1800,3602,0.0,"[0.4, 0.46]"
1,ICC2,Single random raters,0.46,4.46,1800,3600,0.0,"[0.3, 0.57]"
2,ICC3,Single fixed raters,0.54,4.46,1800,3600,0.0,"[0.51, 0.56]"
3,ICC1k,Average raters absolute,0.69,3.24,1800,3602,0.0,"[0.67, 0.72]"
4,ICC2k,Average random raters,0.72,4.46,1800,3600,0.0,"[0.56, 0.8]"
5,ICC3k,Average fixed raters,0.78,4.46,1800,3600,0.0,"[0.76, 0.79]"


### Doublecheck Beaty data
Confirm that this data is what's seen in table 1 of paper - this is correct.

In [29]:
test = pd.read_excel('./../data/Data/Study 1/s1_data_agg.xlsx')
test[[col for col in test if (col.startswith('b_') or col.startswith('r_')) and not col.endswith('_a')]].corr().round(2)

,b_rater1,b_rater2,b_rater3,b_rater4,b_cbowu,b_cbows,b_cboww,b_tasa,b_glov,r_rater1,r_rater2,r_rater3,r_rater4,r_cbowu,r_cbows,r_cboww,r_tasa,r_glov
b_rater1,1.00,0.65,0.54,0.73,0.46,0.32,0.43,0.30,0.26,0.29,0.27,0.36,0.37,0.37,0.32,0.36,0.25,0.37
b_rater2,0.65,1.00,0.59,0.74,0.48,0.37,0.45,0.27,0.24,0.38,0.38,0.48,0.43,0.40,0.42,0.43,0.35,0.42
b_rater3,0.54,0.59,1.00,0.73,0.38,0.45,0.36,0.22,0.21,0.37,0.39,0.51,0.48,0.36,0.39,0.37,0.29,0.35
b_rater4,0.73,0.74,0.73,1.00,0.45,0.38,0.39,0.22,0.26,0.43,0.41,0.54,0.51,0.49,0.45,0.50,0.37,0.45
b_cbowu,0.46,0.48,0.38,0.45,1.00,0.76,0.84,0.36,0.80,0.17,0.17,0.31,0.31,0.42,0.39,0.48,0.28,0.43
b_cbows,0.32,0.37,0.45,0.38,0.76,1.00,0.69,0.32,0.57,0.10,0.13,0.23,0.21,0.35,0.34,0.44,0.19,0.40
b_cboww,0.43,0.45,0.36,0.39,0.84,0.69,1.00,0.41,0.71,0.09,0.10,0.20,0.21,0.41,0.40,0.46,0.17,0.45
b_tasa,0.30,0.27,0.22,0.22,0.36,0.32,0.41,1.00,0.31,-0.05,0.06,0.08,0.10,0.10,0.13,0.15,0.02,0.12
b_glov,0.26,0.24,0.21,0.26,0.80,0.57,0.71,0.31,1.00,0.07,0.08,0.19,0.22,0.37,0.31,0.39,0.22,0.33
r_rater1,0.29,0.38,0.37,0.43,0.17,0.10,0.09,-0.05,0.07,1.00,0.65,0.74,0.74,0.32,0.33,0.32,0.33,0.29


## Normalize MOTES Pilot Data

Score on multi-point scale.

MOTES is related to the "Measuring Original Thinking in Elementary Students: A Text-Mining Approach" (IES #R305A200519). This dataset is very, very new and not yet ready to share. Hopefully it is open access by the time our first LLM paper is published, until then I think we can share with researchers by request (contact <peter.organisciak@du.edu> and/or <selcuk.acar@unt.edu>)

In [30]:
#src = 'motesp'
#motes_pilot = pd.read_csv(os.path.join(base_dir, 'motes_pilot_gt_scores.csv')).rename(columns=dict(D='rater1', K='rater2', T='rater3', ID='participant'))
#motes_pilot['response_num'] = None
#motes_pilot['type'] = motes_pilot.task.apply(lambda x: x.split('_')[0]).replace({'g1':'uses', 'g2':'instances', 'g3': 'consequences', 'g4':'completion'})
#motes_pilot = prep_general(motes_pilot, src, include_stats=False, return_full=True, include_rater_std=include_rater_std)
## Normalize question styles
#motes_pilot.question = (motes_pilot.question.str.replace("What would be a surprising", "What is a surprising", regex=True)
#                            .str.replace("Think of an example of something that is (.*)", "What is a surprising example of something \\1?", regex=True)
#                            .str.replace("What would happen if (.*)", "What would be a surprising consequence if \\1", regex=True)
#)
#comprows = (motes_pilot.type == 'completion')
#motes_pilot.loc[comprows, 'question'] = motes_pilot.loc[comprows, 'question'].str.replace("^(.*)$", 'Complete this sentence in a surprising way: "\\1..."', regex=True)
#datasets[src] = motes_pilot[returncols]
#
#simple_stats(motes_pilot[motes_pilot.id.str.contains('_g1_')], ['rater1', 'rater2', 'rater3'])

## Process MOTES Main Data

This is the post-pilot data. As with the pilot data, this dataset is not yet released, but please reach out.

There are currently 5 raters, though early analysis had 4, which meant the randomized rounding strategy sometimes went into effect - e.g. 2.25 might become 2.3 or 2.2.

In [31]:
#df = pd.read_csv(os.path.join(base_dir, 'motes_full_gt_scores.csv')).replace(-999, pd.NA).copy()
#df['participant'] = df.Order.astype(str).apply(lambda x: hashlib.md5(x.encode('utf-8')).hexdigest()[:6])

In [ ]:
#src = 'motesf'
#
#corrected = True # use spelling corrected columns
#items = [col.replace('_prompt', '') for col in df.columns if col.startswith('G') and col.endswith('_prompt')]
#
#collector = []
## RESHAPE TO long
#for item in items:
#    subset = df[['participant'] + [col for col in df.columns if col.startswith(item)]].copy()
#    subset.columns = [col.split('_')[-1] for col in subset.columns]
#    subset['game'] = item.split('_')[0]
#    subset['prompt_code'] = item
#    collector.append(subset)
#reshaped = pd.concat(collector)
## remove non-responses
#reshaped = reshaped[~reshaped.raw.isna()]
## restore original wording in the test
#reshaped.prompt =reshaped.prompt.str.replace('light bulbs', 'lightbulb').str.replace('hat cap', 'hat').str.replace('soccer ball', 'ball').str.replace('lead pencil', 'pencil').str.replace('spoons', 'spoon')
#
## add display order
#displayorder = df[['participant'] + [col for col in df.columns if 'DO' in col]]
#displayorder = displayorder.melt(id_vars='participant', value_name='prompt_code')
#displayorder['response_num'] = displayorder.variable.apply(lambda x:x[-1])
#reshaped = reshaped.merge(displayorder[['participant', 'prompt_code', 'response_num']])
## use spelling corrected response, unless set otherwise
#reshaped = reshaped.rename(columns={('corrected' if corrected else 'raw'):'response'})
#reshaped['type'] = reshaped['game'].replace({'G1':'uses', 'G2': 'instances', 'G3':'completion'})
#
#completion_ref = {
#    "playground": "When the friends met on the playground...",
#    "school bus": "When I got on the school bus...",
#    "games": "At a sleepover we...",
#    'library': "When the kids were in the library...",
#    'lecture': "When the teacher was talking...",
#    'phone': "My friend called me on the phone to tell me...",
#    'rain': "It started raining and...",
#    'closet': "When I opened my closet...",
#    'lunchroom': "When I was at lunch..."
#}
#reshaped.loc[reshaped.type == 'uses', 'question'] = reshaped.loc[reshaped.type == 'uses', 'prompt'].apply(lambda x: f"What is a surprising use for a {x.upper()}?")
#reshaped.loc[reshaped.type == 'instances','question'] = reshaped.loc[reshaped.type == 'instances', 'prompt'].apply(lambda x: f"What is a surprising example of something {x.upper()}?")
#reshaped.loc[(reshaped.type == 'completion'), 'question'] = reshaped.loc[reshaped.type == 'completion', 'prompt'].replace(completion_ref).str.replace("(.*)", 'Complete this sentence in a surprising way: "\\1"...', regex=True)
#
#datasets[src] = prep_general(reshaped, src,
#                             rater_cols=[col for col in reshaped if 'score' in col.lower()],
#                             include_rater_std=include_rater_std, inputrange=(1,5))
#
## This is code for a separate purpose - exporting spell corrected and uncorrected data
#if False:
#    #corrected_res = datasets[src]
#    #uncorrected = datasets[src]
#    participant_info = df[['Grade', 'Age at MOTES', 'participant']]
#    participant_info.columns = ['grade', 'age', 'participant']
#    participant_info.grade = participant_info.grade.apply(lambda x: int(x[0]))
#    participant_info.participant = src + participant_info.participant
#    spelldata = uncorrected.rename(columns={'response':'raw'}).merge(corrected_res.rename(columns={'response':'corrected'}), on=[col for col in uncorrected.columns if col not in ['id', 'response']]).merge(participant_info)
#    spelldata.to_csv(base_dir / 'motes_full_spelldata.csv')
#
#datasets[src].sample()

## Combine data and basic stats

Check correlation among ratings for responses which have been submitted more than once. Here, I sample one rating vs mean of all the ratings for a duplicate response. This contextualizes the max what a model might be able to do - if humans can't agree (sometimes with themselves!), then it would be impossible for a model to do so.

In [32]:
#@markdown Average rating-to-rating correlation on duplicates
# run multiple times with different samples
from tqdm import trange
corrs = []
for i in trange(1000):
    check_dupe_corr = pd.concat(datasets.values()).sample(frac=1, random_state=i**2)
    just_duped = check_dupe_corr[check_dupe_corr[['prompt', 'response']].duplicated(keep=False)]
    first = just_duped.drop_duplicates(['prompt', 'response'], keep='first')
    last = just_duped.drop_duplicates(['prompt', 'response'], keep='last')
    merged = first.merge(last[['prompt', 'response', 'target']], how='inner', on=['prompt', 'response'])
    corr = merged[['target_x', 'target_y']].corr().values[0,1]
    corrs.append(corr)
print("\nAverage correlation among duplicates", sum(corrs)/len(corrs))

100%|██████████| 1000/1000 [00:42<00:00, 23.27it/s]


Average correlation among duplicates 0.8316375642575191


In [47]:
print(f'Number of datasets combined: {len(datasets)}')
# Number of datapoints in each dataset
for src, df in datasets.items():
    print(f'{src}: {len(df)} data points')
print(f'\nNumber of data points in total: {sum(len(df) for df in datasets.values())}')
merged.head()

#hmsl and motesp datasets not included in the merged dataset: error or not available

Number of datasets combined: 7
dod20: 5490 data points
snbmo09: 4099 data points
hass17: 1093 data points
setal08: 11490 data points
betal18: 2918 data points
snb17: 2372 data points
bs12: 1807 data points

Number of data points in total: 29269


,type,src,question,prompt,response,id,target_x,participant,response_num,target_y
0,uses,dod20,What is a surprising use for a BOTTLE?,bottle,drinking,dod20_bottle-7d11bf,1.0,dod2021,1,1.0
1,uses,snb17,What is a surprising use for a BOX?,box,sled,snb17_box-24a76f,1.3,snb17104,None,1.3
2,uses,dod20,What is a surprising use for a TIRE?,tire,doorstop,dod20_tire-45cef6,3.0,dod2053,1,3.0
3,uses,dod20,What is a surprising use for a PANTS?,pants,scarecrow,dod20_pants-7e08c6,2.5,dod2068,4,2.5
4,instances,setal08,What is a surprising thing that makes a NOISE?,noise,hair dryer,setal08_5.0-2e313a,1.3,setal0819,25.0,1.3


In [50]:
merged.dtypes

type             object
src              object
question         object
prompt           object
response         object
id               object
target_x        float64
participant      object
response_num     object
target_y        float64
dtype: object

In [54]:
#@markdown Average rating-to-mean(other ratings) correlation on duplicates
#corrs = []
#for i in trange(1000):
#    check_dupe_corr = pd.concat(datasets.values()).sample(frac=1)
#    means_of_dupes = check_dupe_corr[check_dupe_corr[['prompt', 'response']].duplicated()].groupby(['prompt','response'], as_index=False).target.mean().round(2)
#    print(means_of_dupes['prompt'].unique())
#    print(means_of_dupes['response'].unique())
#    corr = check_dupe_corr.merge(means_of_dupes[['prompt', 'response', 'target']], on=['prompt', 'response']).corr().loc['target_x', 'target_y']
#    corrs.append(corr)
#print("\nAverage correlation among duplicates", sum(corrs)/len(corrs))

Merge ratings for items with duplicates, so that a response that has been rated multiple times has the average of all instances as it's ground truth.

In [55]:
data = pd.concat(datasets.values()).sample(frac=1, random_state=1234)

# Fix prompt grammar and spelling
data.question = data.question.str.replace('a (PANTS)', r'\1', regex=True).str.replace('LIGHTBULB', 'LIGHT BULB')
#data.prompt = data.prompt.str.replace('lightbulb', 'light bulb')

#@markdown save all data before deduplication for easy access later?
save_all = True #param {type:'boolean'}
save_location = './../data/all_data.csv' #@param {type:'raw'}
if save_all:
    data.to_csv(save_location)

#@markdown For generating splits - focus just on aut?
aut_only = False #@param {type:'boolean'}
if aut_only:
    data = data[data.type == 'uses']

drop_missing = True #@param {type:'boolean'}
if drop_missing:
    data = data[~data.target.isna()]
print(f"Pre-dedupe data size is {len(data)} items")
print(f'# of unique participants is {len(data.participant.unique())}')
print("# of unique prompts", len(data.prompt.unique()))

display(data['src'].value_counts())

combine_gt_for_dupes = True #@param {type:'boolean'}
if combine_gt_for_dupes:
    dupe_means = data.groupby(['prompt', 'response'], as_index=False).target.aggregate(['count', 'mean', 'std']).reset_index().round(1)
    dupe_means = dupe_means[dupe_means['count'] > 1]
    data = data.merge(dupe_means, how='left')
    data.loc[~data['mean'].isna(), 'target'] = data['mean'].dropna() # set target to be mean
    data = data.drop(columns=['mean', 'std'])

ensure_no_dupes = True #@param {type: 'boolean'}
deduplication_strategy = "Uncased Match" #@param ["Exact", "Uncased Match"]
og_size = len(data)
if ensure_no_dupes:
    if deduplication_strategy == "Uncased Match":
        data['response_lower'] = data.response.str.lower()
        data = data.drop_duplicates(['prompt', 'response_lower'])
        data = data.drop('response_lower', axis=1)
    elif deduplication_strategy == "Exact":
        data = data.drop_duplicates(['prompt', 'response'])
    print(f"Dropped {og_size-len(data)} duplicate items. {100*(og_size-len(data))/og_size:.2f}%")
    print(f"Final data size is {len(data)} items")

print('\nproportion of representation by each prompt')
print((data.prompt.value_counts() / len(data)).multiply(100).round(2))

print("\nNumber of datasets with each prompt")
print(data[['prompt', 'src']].drop_duplicates().prompt.value_counts())

Pre-dedupe data size is 29269 items
# of unique participants is 1039
# of unique prompts 16


src
setal08    11490
dod20       5490
snbmo09     4099
betal18     2918
snb17       2372
bs12        1807
hass17      1093
Name: count, dtype: int64

Dropped 8494 duplicate items. 29.02%
Final data size is 20775 items

proportion of representation by each prompt
prompt
brick       19.52
box         13.01
knife       10.48
rope         9.86
noise        8.80
shrank       7.31
no sleep     7.31
round        6.38
bottle       3.35
book         2.35
table        2.23
pants        2.13
tire         1.99
fork         1.96
shoe         1.69
shovel       1.63
Name: count, dtype: float64

Number of datasets with each prompt
prompt
brick       5
rope        3
box         3
knife       2
bottle      2
pants       1
shrank      1
no sleep    1
noise       1
round       1
tire        1
book        1
fork        1
shoe        1
shovel      1
table       1
Name: count, dtype: int64


In [56]:
from scipy.stats import entropy
data.groupby('prompt').target.apply(entropy).sort_values().round(3).to_dict()

{'shovel': 5.759,
 'shoe': 5.78,
 'fork': 5.946,
 'tire': 5.959,
 'pants': 6.03,
 'table': 6.051,
 'book': 6.135,
 'bottle': 6.455,
 'round': 7.134,
 'no sleep': 7.277,
 'shrank': 7.283,
 'noise': 7.476,
 'rope': 7.554,
 'knife': 7.627,
 'box': 7.846,
 'brick': 8.233}

## Save Dataset

In [57]:
name = 'gt_main_with_dupes' #@param {type:'string'}
random_seed = 987 #@param {type:'number'}

# ensure subfolder to avoid rm'ing something dumb
data_dir = os.path.join('data', name)
!rm -rf $data_dir

print(f'####{name}')
print(f"- Data size is {len(data)} items")
os.makedirs(data_dir, exist_ok=True)
splits = {'train': 80, 'val': 5, 'test': 15} #@param {type: 'raw'}
print('seed', random_seed)
print('targetsplits', splits)
#markdown ### Data to Include

#@markdown ### Parameters
#@markdown If splitting data by participant, individuals will be entirely in a single split.
#@markdown If paired with ensure_no_dupes, the test is split out, then the train data is deduped
#@markdown so no test data is seen. This will imbalance the split sizes from expectation.
split_by_participant = False #@param {type: 'boolean'}
#@markdown Split where test data has different prompts than train.
split_by_prompt = False #@param {type: 'boolean'}
try:
    assert not (split_by_participant and split_by_prompt)
except:
    raise AssertionError("Can't split by participant *and* prompt")

print(f'split_by_part: {split_by_participant}; split_by_prompt: {split_by_prompt}')

#@markdown `dry_run`: don't save outputs
dry_run = False #@param {type: 'boolean'}

examples = dict()

def fingerprint(x):
    x = x.lower()
    x = "".join([c for c in x if c.isalpha()])
    return x

if split_by_participant or split_by_prompt:
    try:
        assert set(examples.keys()) == {'test', 'train', 'val'}
    except:
        raise Exception('split_by_participant or split_by_prompt requires test/train/val names')

    sample = dict()
    splitcol = 'participant' if split_by_participant else 'prompt'
    if split_by_participant:
        unique_vals = data[splitcol].unique().tolist()

        sample['train'], sample['test'] = train_test_split(unique_vals, train_size=splits['train']/sum(splits.values()), shuffle=True, random_state=random_seed)
        sample['val'], sample['test']  = train_test_split(sample['test'], train_size=splits['val']/(splits['val']+splits['test']), shuffle=True, random_state=random_seed)
    elif split_by_prompt:
        # most common prompts
        promptn = data.prompt.value_counts() / len(data)
        # put aside the n most plentiful prompts, and add them to train later
        sample['train'] = promptn[:4].index.tolist()
        # recalc, to use splits based on the rest of the data
        #remaining = data[~data.prompt.isin(sample['train'])]
        #promptn = remaining.prompt.value_counts() / len(remaining)
        promptn = promptn[~promptn.index.isin(sample['train'])]

        # fill val and test, then add remainder to train
        pool = promptn.sample(frac=1, random_state=random_seed).cumsum()
        sample['val'] = pool[(pool-promptn.mean()/2) < splits['val']/100].index.tolist()
        promptn = promptn[~promptn.index.isin(sample['val'])]

        pool = promptn.sample(frac=1, random_state=random_seed).cumsum()
        sample['test'] = pool[(pool-promptn.mean()/2) < splits['test']/100].index.tolist()
        promptn = promptn[~promptn.index.isin(sample['test'])]

        sample['train'] += promptn.index.tolist()
        print('train:', sample['train'])
        print('val:', sample['val'])
        print('test:', sample['test'])

    for splittype in splits.keys():
        examples[splittype] = data[data[splitcol].isin(sample[splittype])].to_dict(orient='records')

    if ensure_no_dupes and split_by_participant: #split by prompt is already deduped
        exclude_list = [fingerprint(x['prompt']+x['response']) for x in examples['test']+examples['val']]
        examples['train_og'] = examples['train']
        examples['train'] = [x for x in examples['train'] if fingerprint(x['prompt']+x['response']) not in exclude_list]

else:
    # this approach allows custom split names
    data_shuff = data.sample(frac=1, random_state=random_seed)
    n = 0
    nd = len(data_shuff)
    for k, v in splits.items():
        m = np.rint(n + (nd * v)/100).astype(int)
        examples[k] = data_shuff.iloc[n:m].to_dict(orient='records')
        n = m

sizes = [len(examples[splittype]) for splittype in splits.keys()]
print("Final split sizes:", [np.round(100*x/sum(sizes), 1) for x in sizes])

if not dry_run:
    for split_type, split in splits.items():
        os.makedirs(os.path.join(data_dir, split_type), exist_ok=True)

        for item in examples[split_type]:
            with open(os.path.join(data_dir, split_type, f"{item['id']}.json"), mode='w') as f:
                json.dump(item, f)

    !tar -czf "{name}.tar.gz" data/${name}
    !mv "{name}.tar.gz" "{base_dir}"

'rm' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


####gt_main_with_dupes
- Data size is 20775 items
seed 987
targetsplits {'train': 80, 'val': 5, 'test': 15}
split_by_part: False; split_by_prompt: False
Final split sizes: [80.0, 5.0, 15.0]


tar: : Couldn't visit directory: No such file or directory
tar: Error exit delayed from previous errors.
'mv' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


# Dataset Reference

####gt_main2

This is the main split in the first LLM paper.

- Data size is 20202 items
- seed 987
- targetsplits {'train': 80, 'val': 5, 'test': 15}
- split_by_part: False; split_by_prompt: False
- Final split sizes: [80.0, 5.0, 15.0]
- (gt_main_std *should* be identical, with stdev included, but I haven't
doublechecked)

#### gt_main_with_dupes

- same context as above, but not deduplicated. This is a data leakage context, which should be avoided, though to compare with past work we trained one such model.

####gt_byparticipant
- Data size is 20202 items
- seed 987
- targetsplits {'train': 80, 'val': 5, 'test': 15}
- split_by_part: True; split_by_prompt: False
- Final split sizes: [80.7, 4.7, 14.6]

####gt_byprompt

This the the split used for having different prompts between test and train in the first LLM paper.

- Data size is 20202 items
- seed 987
- targetsplits {'train': 79, 'val': 4, 'test': 17}
- split_by_part: False; split_by_prompt: True
- train: ['brick', 'box', 'knife', 'rope', 'book', 'table', 'tire', 'fork', 'ball', 'pencil', 'lightbulb', 'shoe', 'hat', 'sock', 'toothbrush', 'backpack']
- val: []
- test: ['paperclip', 'spoon', 'bottle', 'shovel', 'pants']
- Final split sizes: [83.2, 0.0, 16.8]

####all

This is the condition for training the final model, for use in the Open Creativity Scoring system. It was *not* deduped for training to avoid data leakeage, since it's for applied use and that performance would be desired.

- Data size is 27217 items
- seed 987
- targetsplits {'train': 94, 'val': 1, 'test': 5}
- split_by_part: False; split_by_prompt: False
- Final split sizes: [94.0, 1.0, 5.0]

####gt_alltests2

This is the condition which includes consequences, instances, and complete the sentence. It may grow outdated as I develop a format for training with this data.

It doesn't have test/train, rather multiple numbered groups so that an ensemble can be trained.

- Data size is 31567 items
- seed 987
- targetsplits {'group1': 32, 'group2': 32, 'group3': 32, 'val': 4}
- split_by_part: False; split_by_prompt: False
- Final split sizes: [32.0, 32.0, 32.0, 4.0]

In [ ]:
#dprint("All gt options")
#print([x.stem.split('.')[0] for x in base_dir.glob('*tar.gz')])

### Combined data can be loaded directly from all_data.csv now

In [59]:
print(f'Length of data: {len(data)}')
data.head()

Length of data: 20775


,type,src,question,prompt,response,id,target,participant,response_num,index,count
0,uses,dod20,What is a surprising use for PANTS?,pants,fashion,dod20_pants-160974,1.0,dod2037,6,14274.0,2.0
1,consequences,setal08,What would be a surprising consequence if EVER...,shrank,couldn't harvest food,setal08_6.0-5d7c8c,1.3,setal08150,7.0,NaN,NaN
2,consequences,setal08,What would be a surprising consequence if PEOP...,no sleep,people would not need to eat the regular healt...,setal08_3.0-5d36eb,1.0,setal08106,2.0,NaN,NaN
3,uses,betal18,What is a surprising use for a ROPE?,rope,use it to tie up a bicycle,betal18_rope-1dd6b7,1.9,betal182093,None,NaN,NaN
4,uses,betal18,What is a surprising use for a ROPE?,rope,steps,betal18_rope-672857,1.3,betal182076,None,NaN,NaN


In [60]:
# save data cleaned (-9000 samples)
data.to_csv('./../data/cleaned_all_data.csv')